In [1]:
import os

In [2]:
os.chdir('../')
%pwd

'/home/paladin/Downloads/Facial_Impression_Recognition_Calassification'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    local_train_dir: Path
    local_val_dir: Path
    local_test_dir: Path
    local_train_file: Path
    local_val_file: Path
    local_test_file: Path
    local_target_file: Path
    local_preprocessor_file: Path

In [4]:
import os
import sys
import pandas as pd
from cnnClassifier.exception import CustomException
from cnnClassifier.logger import logging

from cnnClassifier.constants import *
from cnnClassifier.utils import pixel_to_matrix, save_object, create_directories, read_yaml

2023-07-26 16:32:11.243402: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-26 16:32:11.313774: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-26 16:32:11.314994: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-26 16:32:12.606574: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
class configurationManeger:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH,
                 secret_filepath = SECRET_FILE_PATH,                 
                 params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath) 
        self.secret = read_yaml(secret_filepath)        
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation             

        create_directories([config.root_dir, config.local_train_dir, config.local_val_dir, config.local_test_dir])

        data_trnsformation_config = DataTransformationConfig(
            root_dir = config.root_dir,
            local_train_dir = config.local_train_dir,
            local_val_dir = config.local_val_dir, 
            local_test_dir = config.local_test_dir,
            local_train_file = self.config.data_ingestion.local_train_file,
            local_val_file = self.config.data_ingestion.local_val_file,
            local_test_file = self.config.data_ingestion.local_test_file,
            local_target_file = config.local_target_file,
            local_preprocessor_file = config.local_preprocessor_file

        )

        return data_trnsformation_config

In [6]:
import pandas as pd
from sklearn.preprocessing import FunctionTransformer

from keras.utils import to_categorical
from sklearn.pipeline import Pipeline

In [7]:
from keras.preprocessing.image import save_img

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def get_data_transformer_object(self):
        try:            
            face_recognition_pipeline = Pipeline(
                steps=[
                    ('pixel_to_image', 
                    FunctionTransformer(pixel_to_matrix, kw_args={'img_height':48, 'img_width':48}))
                ]
            )
            return face_recognition_pipeline            
        
        except Exception as e:
            raise CustomException(e, sys)
    
    
    def initiate_data_transformation(self):
        if not os.path.exists(self.config.local_train_file):
            logging.info(f"WARNING: {self.config.local_train_file} does not exist!")
        
        elif not os.path.exists(self.config.local_val_file):
            logging.info(f"WARNING: {self.config.local_val_file} does not exist!")   

        elif not os.path.exists(self.config.local_test_file):
            logging.info(f"WARNING: {self.config.local_test_file} does not exist!")              
        
        else:  
            
            train_df = pd.read_csv(self.config.local_train_file)
            val_df = pd.read_csv(self.config.local_val_file)
            test_df = pd.read_csv(self.config.local_test_file)
            logging.info('Read train, validation and test data completed')

            logging.info("Obtaining preprocessing object")
            preprocessing_obj=self.get_data_transformer_object() 
            pixel_column = 'pixels'
            target_column_name = 'emotion'             

            logging.info(f"Applying preprocessing object on train, validation and test dataframes")
            input_train_arr = preprocessing_obj.fit_transform(train_df[pixel_column])
            for i in range(len(input_train_arr)):
                path_file = os.path.join(self.config.local_train_dir, str(i)+'.png')
                save_img(path_file, input_train_arr[i])
            logging.info(f"Train set is saved as .png")         
            
            input_val_arr = preprocessing_obj.fit_transform(val_df[pixel_column])  
            for i in range(len(input_val_arr)):
                path_file = os.path.join(self.config.local_val_dir, str(i)+'.png')
                save_img(path_file, input_val_arr[i])         
            logging.info(f"Validation set is saved as .png")
            
            input_test_arr = preprocessing_obj.transform(test_df[pixel_column]) 
            for i in range(len(input_test_arr)):
                path_file = os.path.join(self.config.local_test_dir, str(i)+'.png')
                save_img(path_file, input_test_arr[i])  
            logging.info(f"Test set is saved as .png")
            

            logging.info(f"Changing target variable format to match with keras")
            target_train_arr = to_categorical(train_df[target_column_name], 7)   
            target_val_arr = to_categorical(val_df[target_column_name], 7)  
            target_test_arr = to_categorical(test_df[target_column_name], 7) 
            
            logging.info(f"Saving target variables as pickel file")
            target_variable = {'train': target_train_arr, 'validation': target_val_arr, 'test': target_test_arr }           
            save_object(path=self.config.local_target_file, obj=target_variable)
            
            logging.info('Saved preprocessing object')
            save_object(path=self.config.local_preprocessor_file, obj=preprocessing_obj)

            

In [8]:
try:
    config = configurationManeger()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.initiate_data_transformation()

except Exception as e:
    raise CustomException(e, sys)

In [9]:
pic = data.X_train[1]
pic.shape

NameError: name 'data' is not defined

In [ ]:
import matplotlib.image as mpimg

mpimg.imwrite('name.png', pic)

img = mpimg.imread('name.png')
img.shape

AttributeError: module 'matplotlib.image' has no attribute 'imwrite'

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import save_img

pic = data.X_train[1]
pic.shape
print(pic.shape)
save_img('name1.png', pic)
img = load_img('name.png')
img_array = img_to_array(img)
print(img_array.shape)

(48, 48, 3)
(48, 48, 3)


In [ ]:
img_array

array([[[151., 151., 151.],
        [150., 150., 150.],
        [147., 147., 147.],
        ...,
        [129., 129., 129.],
        [140., 140., 140.],
        [120., 120., 120.]],

       [[151., 151., 151.],
        [149., 149., 149.],
        [149., 149., 149.],
        ...,
        [122., 122., 122.],
        [141., 141., 141.],
        [137., 137., 137.]],

       [[151., 151., 151.],
        [151., 151., 151.],
        [156., 156., 156.],
        ...,
        [109., 109., 109.],
        [123., 123., 123.],
        [146., 146., 146.]],

       ...,

       [[188., 188., 188.],
        [188., 188., 188.],
        [121., 121., 121.],
        ...,
        [185., 185., 185.],
        [185., 185., 185.],
        [186., 186., 186.]],

       [[188., 188., 188.],
        [187., 187., 187.],
        [196., 196., 196.],
        ...,
        [186., 186., 186.],
        [182., 182., 182.],
        [187., 187., 187.]],

       [[186., 186., 186.],
        [184., 184., 184.],
        [185., 1